In [1]:
# Instalar Pygame si no está instalado
!pip install pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import pygame
import sys
import random

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [17]:
# Inicializar Pygame
pygame.init()

# Configuración de la pantalla
WIDTH, HEIGHT = 800, 600
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Ping Pong')

# Cargar la imagen de fondo
background_image = pygame.image.load('universo.jpg').convert()
background_image = pygame.transform.scale(background_image, (WIDTH, HEIGHT))

# Cargar las imágenes de las paletas y ajustar su tamaño
paddle_left_image = pygame.image.load('alien1.png').convert_alpha()
paddle_right_image = pygame.image.load('alien2.png').convert_alpha()

# Establecer el ancho deseado de las paletas
paddle_width = 120

# Calcular la altura proporcional para mantener la relación de aspecto
paddle_left_height = int(paddle_left_image.get_height() * (paddle_width / paddle_left_image.get_width()))
paddle_right_height = int(paddle_right_image.get_height() * (paddle_width / paddle_right_image.get_width()))

# Escalar las imágenes de las paletas
paddle_left_image = pygame.transform.scale(paddle_left_image, (paddle_width, paddle_left_height))
paddle_right_image = pygame.transform.scale(paddle_right_image, (paddle_width, paddle_right_height))

# Posiciones iniciales de las paletas
player_position = pygame.Rect(WIDTH - 110, HEIGHT // 2 - paddle_left_height // 2, paddle_width, paddle_left_height)
opponent_position = pygame.Rect(10, HEIGHT // 2 - paddle_right_height // 2, paddle_width, paddle_right_height)

# Cargar la imagen de meteorito y ajustar su tamaño
original_ball_image = pygame.image.load('meteorito.png').convert_alpha()
ball_size = (50, 50)  # Define el tamaño deseado de la imagen de meteorito
ball_image = pygame.transform.scale(original_ball_image, ball_size)
ball_rect = ball_image.get_rect()
ball_rect.center = (WIDTH // 2, HEIGHT // 2)

# Cargar la imagen de explosión y ajustar su tamaño
explosion_image = pygame.image.load('explosion.png').convert_alpha()
explosion_image = pygame.transform.scale(explosion_image, (100, 100))
explosion_rect = explosion_image.get_rect()

# Colores
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

# Configuración de la pelota
ball_speed = 5
ball_dx, ball_dy = ball_speed, ball_speed

# Velocidad y pausa
player_speed = 7
paused = False

# Puntuación
player_score, opponent_score = 0, 0

# Tamaño de la fuente
font_size = 36
font = pygame.font.Font(None, font_size)  # None indica la fuente predeterminada de Pygame

# Tiempo de la explosión
explosion_timer = 0
explosion_duration = 1 * 1000  # 1 segundo en milisegundos

# Función para mostrar el texto en la pantalla
def draw_text(text, font, color, surface, x, y):
    textobj = font.render(text, True, color)
    textrect = textobj.get_rect()
    textrect.center = (x, y)
    surface.blit(textobj, textrect)

# Función para mostrar la explosión
def show_explosion(x, y):
    global explosion_timer, explosion_rect
    explosion_timer = pygame.time.get_ticks()
    explosion_rect.center = (x, y)
    SCREEN.blit(explosion_image, explosion_rect)

# Función principal del juego
def main():
    global ball_dx, ball_dy, player_score, opponent_score, paused, explosion_timer
    clock = pygame.time.Clock()
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_p:
                    paused = not paused

        if not paused:
            keys = pygame.key.get_pressed()
            if keys[pygame.K_UP] and player_position.top > 0:
                player_position.y -= player_speed
            if keys[pygame.K_DOWN] and player_position.bottom < HEIGHT:
                player_position.y += player_speed

            # Movimiento de la pelota
            ball_rect.x += ball_dx
            ball_rect.y += ball_dy

            # Colisión con los bordes de la pantalla
            if ball_rect.top <= 0 or ball_rect.bottom >= HEIGHT:
                ball_dy *= -1
            if ball_rect.left <= 0:
                player_score += 1
                show_explosion(ball_rect.left, ball_rect.centery)
                ball_restart()
            if ball_rect.right >= WIDTH:
                opponent_score += 1
                show_explosion(ball_rect.right, ball_rect.centery)
                ball_restart()

            # Colisión con las paletas
            if ball_rect.colliderect(player_position) or ball_rect.colliderect(opponent_position):
                ball_dx *= -1

            # Movimiento del oponente (opcional)
            if opponent_position.top < ball_rect.y:
                opponent_position.y += player_speed
            if opponent_position.bottom > ball_rect.y:
                opponent_position.y -= player_speed

            # Control de la explosión
            if explosion_timer != 0:
                current_time = pygame.time.get_ticks()
                if current_time - explosion_timer >= explosion_duration:
                    explosion_timer = 0

        # Dibujar todo en la pantalla
        SCREEN.blit(background_image, (0, 0))  # Dibujar el fondo primero
        SCREEN.blit(paddle_left_image, player_position)  # Dibujar la paleta izquierda
        SCREEN.blit(paddle_right_image, opponent_position)  # Dibujar la paleta derecha
        SCREEN.blit(ball_image, ball_rect)  # Mostrar la imagen de la pelota en lugar de la elipse
        pygame.draw.aaline(SCREEN, WHITE, (WIDTH // 2, 0), (WIDTH // 2, HEIGHT))

        draw_text(f'{player_score}', font, WHITE, SCREEN, WIDTH // 2 + 50, 50)
        draw_text(f'{opponent_score}', font, WHITE, SCREEN, WIDTH // 2 - 50, 50)

        # Mostrar la explosión si está activa
        if explosion_timer != 0:
            SCREEN.blit(explosion_image, explosion_rect)

        # Botón de pausa
        pause_button = pygame.Rect(WIDTH - 120, 20, 100, 50)
        pygame.draw.rect(SCREEN, GRAY, pause_button)
        draw_text('PAUSA', font, BLACK, SCREEN, WIDTH - 70, 45)

        if paused:
            draw_text('PAUSADO', font, WHITE, SCREEN, WIDTH // 2, HEIGHT // 2)

        pygame.display.flip()
        clock.tick(60)

# Función para reiniciar la posición de la pelota
def ball_restart():
    global ball_dx, ball_dy
    ball_rect.center = (WIDTH // 2, HEIGHT // 2)
    ball_dx *= -1

# Función de instrucciones (opcional)
def instructions():
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    running = False

        SCREEN.fill(BLACK)
        draw_text('Cómo Jugar', font, WHITE, SCREEN, WIDTH // 2, HEIGHT // 4)
        font_small = pygame.font.Font(None, 28)
        draw_text('Usa las flechas arriba y abajo para mover el alien.', font_small, WHITE, SCREEN, WIDTH // 2, HEIGHT // 2 - 40)
        draw_text('Presiona P para pausar el juego.', font_small, WHITE, SCREEN, WIDTH // 2, HEIGHT // 2)
        draw_text('Presiona ENTER para volver al menú principal.', font_small, WHITE, SCREEN, WIDTH // 2, HEIGHT // 2 + 40)
        pygame.display.flip()

# Menú del juego (opcional)
def menu():
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    main()
                if event.key == pygame.K_a:
                    instructions()

        SCREEN.fill(BLACK)
        draw_text('¡Defiende tu Galaxia!', font, WHITE, SCREEN, WIDTH // 2, HEIGHT // 4)
        draw_text('Presiona ENTER para comenzar', font, WHITE, SCREEN, WIDTH // 2, HEIGHT // 2)
        draw_text('Presiona A para instrucciones', font, WHITE, SCREEN, WIDTH // 2, HEIGHT // 2 + 60)
        pygame.display.flip()

menu()


SystemExit: 